In [4]:
import pandas as pd
import numpy as np
import datetime
import plotly.express as px
import yfinance as yf
import pandas_market_calendars as mcal
from plotly.offline import init_notebook_mode, plot
init_notebook_mode(connected=True)

In [5]:
def create_market_cal(start, end):
    nyse = mcal.get_calendar('NYSE')
    schedule = nyse.schedule(stocks_start, stocks_end)
    market_cal = mcal.date_range(schedule, frequency='1D')
    market_cal = market_cal.tz_localize(None)
    market_cal = [i.replace(hour=0) for i in market_cal]
    return market_cal


def get_data(stocks, start, end):
    def data(ticker):
        df = yf.download(ticker, start=start, end=(end + datetime.timedelta(days=1)))
        df['symbol'] = ticker
        df.index = pd.to_datetime(df.index)
        return df
    datas = map(data, stocks)
    return(pd.concat(datas, keys=stocks, names=['Ticker', 'Date'], sort=True))


def get_benchmark(benchmark, start, end):
    benchmark = get_data(benchmark, start, end)
    benchmark = benchmark.drop(['symbol'], axis=1)
    benchmark.reset_index(inplace=True)
    return benchmark

In [6]:
import requests
from bs4 import BeautifulSoup

In [7]:
import requests
import re

keys = ['XLU', 'XLRE']


headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:83.0) Gecko/20100101 Firefox/83.0"
}


def main(url):
    with requests.Session() as req:
        req.headers.update(headers)
        for key in keys:
            r = req.get(url.format(key))
            print(f"Extracting: {r.url}")
            goal = re.findall(r'etf\\\/(.*?)\\', r.text)
            print(goal)


main("https://www.zacks.com/funds/etf/{}/holding")

Extracting: https://www.zacks.com/funds/etf/XLU/holding
['NEE', 'DUK', 'SO', 'D', 'AEP', 'SRE', 'EXC', 'XEL', 'ED', 'PEG', 'WEC', 'ES', 'AWK', 'EIX', 'DTE', 'AEE', 'ETR', 'FE', 'PPL', 'CEG', 'CMS', 'CNP', 'ATO', 'EVRG', 'LNT', 'AES', 'NI', 'NRG', 'PNW']
Extracting: https://www.zacks.com/funds/etf/XLRE/holding
['AMT', 'PLD', 'CCI', 'EQIX', 'PSA', 'O', 'DLR', 'WELL', 'SBAC', 'SPG', 'VICI', 'AVB', 'WY', 'EQR', 'CBRE', 'EXR', 'ARE', 'DRE', 'VTR', 'MAA', 'ESS', 'IRM', 'CPT', 'PEAK', 'UDR', 'BXP', 'KIM', 'HST', 'REG', 'FRT', 'VNO']


KeyError: 'VPU'

SyntaxError: EOL while scanning string literal (Temp/ipykernel_4164/471075802.py, line 6)

In [18]:
from yahooquery import Ticker

ticker_name = 'INDA US Equity'
ticker = ticker_name[:-10]

t = Ticker(ticker_name)

# sector weightings, returns pandas DataFrame
sector_dict_list = t.fund_sector_weightings[ticker]['sectorWeightings']
dict_sector = {}
for d in sector_dict_list:
    dict_sector.update(d)

df_sector = pd.DataFrame(dict_sector.items(), 
                         columns=['Sector', 'Weight'])\
                        .sort_values('Weight',
                         ascending = False)
df_sector['Weight'] = df_sector['Weight']
df_sector=df_sector.set_index('Sector')
# df_sector

fig = px.bar(df_sector, 
             x=df_sector.index, y='Weight',
             color='Weight',
             hover_data=[df_sector.index, 'Weight'],
             height=400,
             title = ticker_name + ' SECTOR BREAKDOWN',
#              text = 'Weight'
            )
fig.show()

In [9]:
#- How many assests to include in each portfolio
n_assets = 5
#-- How many portfolios to generate
n_portfolios = 1000

#-- Initialize empty list to store mean-variance pairs for plotting
mean_variance_pairs = []

np.random.seed(75)
#-- Loop through and generate lots of random portfolios
for i in range(n_portfolios):
    #- Choose assets randomly without replacement
    assets = np.random.choice(list(daily_returns.columns), n_assets, replace=False)
    #- Choose weights randomly
    weights = np.random.rand(n_assets)
    #- Ensure weights sum to 1
    weights = weights/sum(weights)

    #-- Loop over asset pairs and compute portfolio return and variance
    #- https://quant.stackexchange.com/questions/43442/portfolio-variance-explanation-for-equation-investments-by-zvi-bodie
    portfolio_E_Variance = 0
    portfolio_E_Return = 0
    for i in range(len(assets)):
        portfolio_E_Return += weights[i] * mus.loc[assets[i]]
        for j in range(len(assets)):
            #-- Add variance/covariance for each asset pair
            #- Note that when i==j this adds the variance
            portfolio_E_Variance += weights[i] * weights[j] * cov.loc[assets[i], assets[j]]
            
    #-- Add the mean/variance pairs to a list for plotting
    mean_variance_pairs.append([portfolio_E_Return, portfolio_E_Variance])

NameError: name 'daily_returns' is not defined

In [ ]:

#-- Plot the risk vs. return of randomly generated portfolios
#- Convert the list from before into an array for easy plotting
mean_variance_pairs = np.array(mean_variance_pairs)
risk_free_rate=0 #-- Include risk free rate here for sharpe ratio

#-- Create Plot
fig = go.Figure()
fig.add_trace(go.Scatter(x=mean_variance_pairs[:,1]**0.5, 
                         y=mean_variance_pairs[:,0], 
                      #- Add color scale for sharpe ratio   
                      marker=dict(color=(mean_variance_pairs[:,0]-risk_free_rate)/(mean_variance_pairs[:,1]**0.5), 
                                  showscale=True, 
                                  size=7,
                                  line=dict(width=1),
                                  colorscale="RdBu",
                                  colorbar=dict(title="Sharpe<br>Ratio")
                                 ), 
                      mode='markers'))
#- Add title/labels
fig.update_layout(template='plotly_white',
                  xaxis=dict(title='Annualised Risk (Volatility)'),
                  yaxis=dict(title='Annualised Return'),
                  title='Sample of Random Portfolios',
                  coloraxis_colorbar=dict(title="Sharpe Ratio"))
view rawplot_mv_simple.py hosted with ❤ by GitHub